# Predicting Income

In this lab we'll be using a dataset from kaggle yet again...it's just so fun and rich! We're using the following income dataset where we want to use the other features to predict whether someone is making over $50,000 per year or not.

Primary Goals:

Predict income.
Assignment Specs:

You need to use Naive Bayes and neural networks in your work to answer the question above, but you should explore at least two other models in order to answer the above questions as best you can. You may use multiple neural network models if you like, but I'd encourage you to consider past model types we've discussed.
This dataset has variables of multiple types. So, this should give you an opportunity to explore how neural networks can (or can't) handle data of different types. You may need to one-hot encode the character variables...
Your submission should be built and written with non-experts as the target audience. All of your code should still be included, but do your best to narrate your work in accessible ways.
Again, submit an HTML, ipynb, or Colab link. Be sure to rerun your entire notebook fresh before submitting!

In [1]:
# load in libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [15]:
# load in data
income = pd.read_csv("income_evaluation.csv")
income = income.dropna()
income.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
income.columns = income.columns.str.strip()
# encode 1 as over 50K and 0 as less than 50K
# income['income_binary'] = income['income'].apply(lambda x: 0 if x.strip() == '<=50K' else 1)
# income = income.drop(columns = ["income"])

# Naive Bayes

In [17]:
X = income.drop(columns = ["income"])
y = income[["income"]]

In [ ]:
# define column transformer
ct = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False, handle_unknown='ignore'), make_column_selector(dtype_include=object)),
        ("standardize", StandardScaler(), make_column_selector(dtype_include=np.number))
        
    ],
    remainder="passthrough"
).set_output(transform="pandas")


